## 9/9 기준

In [ ]:
!pip install -qU langchain-openai langchain-huggingface chromadb gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 32.3 MB/s eta 0:

In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
!pip install langgraph -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 11.7 MB/s eta 0:00:00


In [ ]:
from typing import TypedDict, List, Dict, Any, Optional
from pathlib import Path
import os
import shutil
import tempfile
import gdown
import json
import torch

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

## 벡터DB 다운
- vector_db.py
- -> 코랩 기준으로 파일 저장 위치 수정

In [ ]:
def download_drive_folder_to_chroma_db(folder_url: str, target_dir: Path):
    target_dir = Path(target_dir).resolve()
    target_dir.mkdir(parents=True, exist_ok=True)
    with tempfile.TemporaryDirectory() as td:
        gdown.download_folder(url=folder_url, output=td, quiet=False, use_cookies=False)
        entries = [Path(td) / name for name in os.listdir(td)]
        src_root = entries[0] if len(entries) == 1 and entries[0].is_dir() else Path(td)
        for p in src_root.iterdir():
            dst = target_dir / p.name
            if dst.exists():
                shutil.rmtree(dst) if dst.is_dir() else dst.unlink()
            shutil.move(str(p), str(dst))
    if not (target_dir / "chroma.sqlite3").exists():
        raise RuntimeError(f"'chroma.sqlite3'가 없습니다: {target_dir}")

def create_chroma_db():
    HERE = Path(".").resolve()
    FOLDER_URL = "https://drive.google.com/drive/folders/1_paLIqOIeOyozE-wsKuMO4wIiOpaLio9"
    download_drive_folder_to_chroma_db(FOLDER_URL, HERE / "chroma_db")

## retriever.py
- -> 코랩 기준으로 파일 저장 위치 수정

In [ ]:
HERE = "."
DB_DIR = os.path.join(HERE, "chroma_db")
COLLECTION_NAME = "qna_collection"
EMBED_MODEL = "BAAI/bge-m3"
TOP_K = 5
embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    encode_kwargs={"normalize_embeddings": True},
)

def retriever_setting():
    if not os.path.isdir(DB_DIR):
        print("ChromaDB 디렉토리가 없습니다. 구글 드라이브에서 다운로드합니다.")
        create_chroma_db()
    vs = Chroma(
        collection_name=COLLECTION_NAME,
        persist_directory=DB_DIR,
        embedding_function=embeddings,
    )
    retriever = vs.as_retriever(search_kwargs={"k": TOP_K})
    return retriever

## rag2.py & rag.py
- basic_chain_setting... 어떤 파일 함수를 써야 하는지?

In [ ]:
def basic_chain_setting(): # rag2.py
    llm = ChatOpenAI(model="gpt-4o", temperature=0)

    basic_prompt = PromptTemplate.from_template(
        """
    당신은 api 문서 관련 전문 챗봇으로서 사용자의 질문에 정확하고 친절하게 답변해야 합니다.
    아래 제공되는 문서에 없는 내용은 절대 답변에 포함하지 말고, 문서 내에서만 답변 내용을 찾아서 제공하세요.
    만약 사용자 질문이 구글 api 문서에 대한 질문이 아니라면, 아래 문서는 무시하고 일상 질문에 대해서만 답변하세요.

    문서 : {context}

    이전 대화 내역 : {history}

    이번 사용자 질문 : {question}
    """
    )

    basic_chain = basic_prompt | llm | StrOutputParser()

    return basic_chain


def query_setting():
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
        model_kwargs={"response_format": {"type": "json_object"}},
    )

    query_prompt = PromptTemplate.from_template(
        """
       유저의 채팅 히스토리와 현재 질문이 주어집니다.


       **중요**: 이전 대화 맥락을 반드시 고려해서 질문을 생성하세요.
       - 현재 질문이 이전 대화와 연관되어 있다면, 이전 맥락을 포함한 통합된 질문을 만들어주세요.

       - 예: 바로 전에 "People API 연락처 조회"에 대해 이야기하고 나서, "그럼 프로필 수정은?"이라는 질문이 나오면 "People API에서 프로필 수정 방법"으로 통합해주세요.
       - 주의사항: 이전에 "People API 연락첯 조회"에 대해 이야기하고 나서, "Firebase"와 같이 다른 api에 대한 대화 내용이 나온 후 "프로필 수정은?"이라는 질문이 나오면 마지막 대화 맥락에 맞춰서, "Firebase에서 프로필 수정 방법"과 같이 통합해야 합니다.

       - 이전 대화에서 이미 답변이 나온 질문은 생성하지 마세요.
       - 질문은 1개가 될 수도 있고 여러개가 될 수도 있습니다.

       대화 히스토리: {rewritten}

       JSON 반환 형태:
       {{"questions": ["맥락을 고려한 통합 질문 1", "맥락을 고려한 통합 질문 2", ...]}}
       """
    )

    def parse_json(response):
        return json.loads(response.content)  # response.content 사용

    chain = query_prompt | llm | parse_json
    return chain

## langgraph_node.py

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_F1")

In [ ]:
basic_chain = basic_chain_setting()
retriever = retriever_setting()
query_chain = query_setting()


class ChatState(TypedDict, total=False):
    question: str  # 유저 질문
    answer: str  # 모델 답변
    rewritten: str  # 통합된 질문
    queries: List[str]  # 쿼리(질문들)
    search_results: List[str]  # 벡터 DB 검색 결과들
    messages: List[Dict[str, str]]  # 사용자 및 모델의 대화 히스토리


# (1) 사용자 질문 + 히스토리 통합 → 통합된 질문과 쿼리 추출
def extract_queries(state: ChatState) -> ChatState:
    user_text = state["question"]

    # 히스토리에서 최근 몇 개의 메시지를 가져와서 통합 질문을 생성
    messages = state.get("messages", [])

    # 최근 4개 메시지만 사용
    history_tail = messages[-4:] if messages else []
    context = history_tail.copy()

    # 현재 사용자 질문 추가
    context.append({"role": "user", "content": user_text})
    state["rewritten"] = context

    return state


# (2) LLM에게 질문 분리를 시킨다
def split_queries(state: ChatState) -> ChatState:
    rewritten = state.get("rewritten")

    response = query_chain.invoke({"rewritten": rewritten})
    state["queries"] = response["questions"]  # questions 리스트만 저장

    return state


# (3) 벡터 DB 툴 호출
def search_tool(query: str):
    """질문을 바탕으로 벡터 DB에서 결과 검색"""
    return retriever.invoke(query)  # retriever는 DB 검색 로직을 호출


# (4) 기본 답변 생성 노드
def langgraph_node(state: ChatState) -> Dict[str, Any]:
    history = state.get("messages", [])
    """질문에 대한 기본 답변 생성"""
    queries = state["queries"]
    print(f"생성된 질문 리스트 {queries}")
    search_results = []

    # 각 쿼리마다 벡터 DB 검색
    for query in queries:
        print(f"{query} 검색중...")
        results = search_tool(query)
        search_results.append(results)  # 검색된 결과들을 모아서 저장

    # 검색된 결과를 바탕으로 답변 생성
    answer = basic_chain.invoke(
        {
            "question": state["question"],
            "history": history,
            "context": "\n".join([str(res) for res in search_results]),
        }
    ).strip()

    state["search_results"] = search_results
    state["answer"] = answer

    return state  # 답변을 반환

## langgraph_setting_edit.py
- langgraph_setting.py 이건 쓰이는지?

In [ ]:
# 그래프 설정
def graph_setting_edit():
    # LangGraph 정의
    graph = StateGraph(ChatState)

    # 노드 등록
    graph.add_node("extract_queries", extract_queries)  # 질문 통합 + 쿼리 추출 노드
    graph.add_node("split_queries", split_queries)  # 질문 분리 툴
    graph.add_node("basic", langgraph_node)  # 기본 답변 노드

    # 시작 노드 정의
    graph.set_entry_point("extract_queries")

    # 흐름 설정
    graph.add_edge("extract_queries", "split_queries")  # 질문 추출 후 분리
    graph.add_edge("split_queries", "basic")  # 쿼리 분리 후 기본 답변 노드로 넘어감
    graph.add_edge("basic", END)  # 기본 답변 후 종료

    # 그래프 컴파일
    memory = MemorySaver()
    compiled_graph = graph.compile(checkpointer=memory)

    return compiled_graph

## 실행
- main2.py

In [ ]:
graph = graph_setting_edit()


def run_langraph(user_input, config_id, chat_history=None):
    config = {"configurable": {"thread_id": config_id}}

    # chat_history가 None이면 빈 리스트로 초기화
    if chat_history is None:
        chat_history = []

    result = graph.invoke(
        {"messages": chat_history, "question": user_input}, config=config
    )

    return result["answer"]

## test

In [ ]:
dummy_history = [
    {"role": "user", "content": "firebase auth의 인증 설정에서 응답 형식은 어떻게 되나요?"},
    {"role": "assistant", "content": "firebase의 인증 설정 응답은 JSON 형식으로 반환됩니다."},
    {"role": "user", "content": "people api의 프로필 설정 방식에서 응답 형식은 어떻게 되나요?"},
    {"role": "assistant", "content": "people api의 프로필 설정 응답은 JSON 형식으로 반환됩니다."}
]

In [ ]:
response = run_langraph('내가 방금 뭐 물어봤어?', 'test1', dummy_history)

생성된 질문 리스트 ['People API의 프로필 설정 응답 형식은 어떻게 되나요?']
People API의 프로필 설정 응답 형식은 어떻게 되나요? 검색중...


In [ ]:
print(response)

사용자님께서는 방금 "people api의 프로필 설정 방식에서 응답 형식은 어떻게 되나요?"라고 질문하셨습니다.
